# FHIR Digitally Signing FHIR Bundle or QuestionnaireResponse Object

This is a Jupyter Notebook using Python 3.7 and openSSl to create JSON Web Signature (JWS)(see RFC 7515) and attach it to a FHIR Bundle or QuestionnaireResponse resource.

- If the resource is a Bundle use Bundle.signature
- If the resource is a QuestionnaireResponse use its [Signature extension](http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature
)

See enveloped signatures: http://build.fhir.org/signatures.html

*Although self-signed certificates are used for the purpose of these examples, they are not recommended for production systems.*

### Import Libraries

In [56]:
from requests import get, post
from json import dumps, loads
from pathlib import Path
from datetime import datetime
import pytz
from jose import jws  #python JWS package
from base64 import b64encode
from jcs import canonicalize #package for a JCS (RFC 8785) compliant canonicalizer.

### 1. Define the location of the Document Signing Certificate

 - See the Jupyter file [Create_Cert.ipynb]() for how to generate your own self-signed certificate.

In [57]:
certificate_path = Path('example_org_cert') # update this to your folder

### 3. Create JWS to Attach to Bundle or QR (START HERE IF USING THE SAME CERT)

#### 3.01  Auto Timestamp Function

In [58]:
auto_timestamp = False
timezone= 'US/Pacific'

def get_timestamp(timezone):  # Get current time in timezone
  tz = pytz.timezone('US/Pacific')
  current_time = datetime.now(tz)
  iso_timestamp = current_time.isoformat()  # Format as ISO 8601 compliant string
  return(iso_timestamp)
# print(get_timestamp(timezone))

#### 3.1. Prepare Header

 note the base64 DER is Cert PEM file wihout the footer and header and line returns

In [ ]:
cert_pem = certificate_path / 'cert.pem'
der = cert_pem.read_text()
der = der.replace('-----BEGIN CERTIFICATE-----','')
der = der.replace('-----END CERTIFICATE-----','')
der = der.replace('\n','')
# kid_file = certificate_path / 'kid.txt' TODO use pytbon library to extract kid
# kid = kid_file.read_text().replace('\n','')
header = {
    "alg": "RS256",
    "kty": "RS",
    "sigT": get_timestamp(timezone) if auto_timestamp else '2020-10-23T04:54:56.048+00:00',
    "kid": '123345',
    "x5c": [der],
     }
header

{'alg': 'RS256',
 'kty': 'RS',
 'sigT': '2020-10-23T04:54:56.048+00:00',
 'kid': '6751A58479F08DA4B40755284737B4278DF4B7EF',
 'x5c': ['MIIFWjCCA8KgAwIBAgIUPA0MvjjKGU3uLz9rNkQfVkOmkSUwDQYJKoZIhvcNAQELBQAwgZUxCzAJBgNVBAYTAlVTMRMwEQYDVQQIDApDYWxpZm9ybmlhMRIwEAYDVQQHDAlTYXVzYWxpdG8xHTAbBgNVBAoMFEV4YW1wbGUgT3JnYW5pemF0aW9uMRkwFwYDVQQDDBBKb2huIEhhbmNvY2ssIE1EMSMwIQYJKoZIhvcNAQkBFhRqaGFuY29ja0BleGFtcGxlLm9yZzAeFw0yNTA2MTkyMTE5MDFaFw0yNzA2MDkyMTE5MDFaMIGVMQswCQYDVQQGEwJVUzETMBEGA1UECAwKQ2FsaWZvcm5pYTESMBAGA1UEBwwJU2F1c2FsaXRvMR0wGwYDVQQKDBRFeGFtcGxlIE9yZ2FuaXphdGlvbjEZMBcGA1UEAwwQSm9obiBIYW5jb2NrLCBNRDEjMCEGCSqGSIb3DQEJARYUamhhbmNvY2tAZXhhbXBsZS5vcmcwggGiMA0GCSqGSIb3DQEBAQUAA4IBjwAwggGKAoIBgQC0LqHDFizJGtoyA68T1MbbOw+AyV5TXgEE5HDoaihklxPGLUgqYFUy5+Q5ipwqftQzrZBcSuWmB6MI11jo2BzrQZ+NVXa4ipzPxwc1Y3NJ8Q404Tjkio0BW8bLgQpLMeoPJAwIvaP66gIFdkLWuW17CGpRfl1phyx56IcbLeeqJ0VJhiLnJgvtq1E0Tz2/IQqpTMsniFNQ05kp4wguAzzvFKdb9AN4Eb7jLnjY9xylagVLtn86TjnoPV6xT/ZFB/gB0mOsMU4lBwpaWKJawYjIe4aj9xPhs2dnTJ6fIT0pSZYx8uUcQ

### 3.2 Fetch Payload from FHIR Server or Local File

- **set local_file to "True" for local file**
- to preserve meta and id in the fetched payload need to:
   -  run ig-publisher only  ("-i" option, no "-k" or "-n" options)
   -  remove example from the parameter
  

In [60]:
local_file = True
payload_id = 'Bundle-cdex-document-digital-sig-example.json' # can be a Bundle or Questionnaire

def fetch_payload():

  # url = "https://argopatientlist.aidbox.app/fhir/Bundle"
  # url = 'http://test.fhir.org/r4/Bundle'
  url = 'https://hl7.org/fhir/us/davinci-cdex'

  username = "basic"
  password = "secret"
  headers = {"Accept": "application/fhir+json" , "Content-Type": "application/fhir+json"}

  r = get(f'{url}/{payload_id}', auth=(username, password), headers = headers)
  my_obj = r.json()
  # print("="*80)
  # print("STATUS: ",r.status_code)

  # print("="*80)
  # print("HEADERS:\n")
  # for k,v in r.headers.items():
  #     print(f'{k} = {v}')
  # print("="*80)
  # print("BODY:\n")
  # print(dumps(my_obj,indent=2))
  return(my_obj)


if local_file: 
  in_path = '/Users/ehaas/Documents/FHIR/davinci-ecdx/output'

  path = Path() / in_path / payload_id
  my_obj =loads(path.read_text())
else:
  my_obj = fetch_payload()
print(dumps(my_obj,indent=2))

{
  "resourceType": "Bundle",
  "id": "cdex-document-digital-sig-example",
  "meta": {
    "extension": [
      {
        "url": "http://hl7.org/fhir/StructureDefinition/instance-name",
        "valueString": "CDEX Document with Digital Signature Example"
      },
      {
        "url": "http://hl7.org/fhir/StructureDefinition/instance-description",
        "valueMarkdown": "Digital signature example showing how it is used to sign a FHIR Document.  The CDEX use case would be the target resource in response to a Task-based request where a digital signature was required.  If no signature was required, the response would typically be in the form of an individual resource."
      }
    ],
    "profile": [
      "http://hl7.org/fhir/us/davinci-cdex/StructureDefinition/cdex-signature-bundle"
    ]
  },
  "identifier": {
    "system": "urn:ietf:rfc:3986",
    "value": "urn:uuid:c173535e-135e-48e3-ab64-38bacc68dba8"
  },
  "type": "document",
  "timestamp": "2021-10-25T20:16:29-07:00",
  "entr

#### 3.2.1 Prepare Payload

The payload is the base64_url form of the canonicalized version of the resource before attaching the signature
 

#####  Canonicalize the resource using IETF JSON Canonicalization Scheme (JCS) before adding the signature element

- Remove the id, meta, and signature elements if present before canonicalization
- Note keep the the generated narrative if present

In [61]:
def  pop_element(resource, element):
  try:
    my_element = resource.pop(element) # remove element
    return my_element 
  except KeyError:
    pass


if my_obj['resourceType'] in ['Bundle', 'QuestionnaireResponse']:
  my_obj_id = pop_element(my_obj, 'id')
  my_obj_meta = pop_element(my_obj, 'meta')
  my_obj_old_sig = pop_element(my_obj, 'signature')
else:
  print('Not a Bundle or QuestionnaireResponse')


if my_obj['resourceType'] == 'QuestionnaireResponse':
  try:
    for i, extension in enumerate(my_obj['extension']):
       if extension['url'] == 'http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature':
          my_obj_signature_ext = my_obj['extension'].pop(i) # remove element
    if i == 0:
      my_obj.pop('extension') # remove extension if empty
  except KeyError:
    print('No signature extension found')

payload = canonicalize(my_obj)
payload, len(payload)

(b'{"entry":[{"fullUrl":"urn:uuid:17a80a8d-4cf1-4deb-a1fd-2db1130e5f76","resource":{"attester":[{"mode":"legal","party":{"display":"Example Practitioner","reference":"urn:uuid:0820c16d-91de-4dfa-a3a6-f140a516a9bc"},"time":"2021-10-25T20:16:29-07:00"}],"author":[{"display":"Example Practitioner","reference":"urn:uuid:0820c16d-91de-4dfa-a3a6-f140a516a9bc"}],"date":"2021-10-25T20:16:29-07:00","encounter":{"display":"Example Encounter","reference":"urn:uuid:5ce5c83a-000f-47d2-941c-039358cc9112"},"id":"17a80a8d-4cf1-4deb-a1fd-2db1130e5f76","resourceType":"Composition","section":[{"entry":[{"reference":"urn:uuid:014a68ec-d691-49e0-b980-91b0d924e570"}],"title":"Active Condition 1"}],"status":"final","subject":{"display":"Example Patient","reference":"urn:uuid:970af6c9-5bbd-4067-b6c1-d9b2c823aece"},"text":{"div":"<div xmlns=\\"http://www.w3.org/1999/xhtml\\"><a name=\\"Composition_17a80a8d-4cf1-4deb-a1fd-2db1130e5f76\\"> </a><p class=\\"res-header-id\\"><b>Generated Narrative: Composition 17a8

##### Then base64_url the payload entry

note this step is combined with 3.3 below using the jws.sign method.

#### 3.3 Create Signature using private key and the RS256 algorithm to get the JWS compact serialization format

note the signature is displayed with the parts labeled and separated with line breaks for easier viewing.

In [62]:
private_key_path = certificate_path / 'private-key.pem'
private_key = private_key_path.read_text()
private_key

signature = jws.sign(payload,private_key,algorithm='RS256',headers=header)

labels = ['header', 'payload', 'signature']
for i,j in enumerate(signature.split('.')):
    print(f'{labels[i]}:\n{j}\n')

header:
eyJhbGciOiJSUzI1NiIsImtpZCI6IjY3NTFBNTg0NzlGMDhEQTRCNDA3NTUyODQ3MzdCNDI3OERGNEI3RUYiLCJrdHkiOiJSUyIsInNpZ1QiOiIyMDIwLTEwLTIzVDA0OjU0OjU2LjA0OCswMDowMCIsInR5cCI6IkpXVCIsIng1YyI6WyJNSUlGV2pDQ0E4S2dBd0lCQWdJVVBBME12ampLR1UzdUx6OXJOa1FmVmtPbWtTVXdEUVlKS29aSWh2Y05BUUVMQlFBd2daVXhDekFKQmdOVkJBWVRBbFZUTVJNd0VRWURWUVFJREFwRFlXeHBabTl5Ym1saE1SSXdFQVlEVlFRSERBbFRZWFZ6WVd4cGRHOHhIVEFiQmdOVkJBb01GRVY0WVcxd2JHVWdUM0puWVc1cGVtRjBhVzl1TVJrd0Z3WURWUVFEREJCS2IyaHVJRWhoYm1Odlkyc3NJRTFFTVNNd0lRWUpLb1pJaHZjTkFRa0JGaFJxYUdGdVkyOWphMEJsZUdGdGNHeGxMbTl5WnpBZUZ3MHlOVEEyTVRreU1URTVNREZhRncweU56QTJNRGt5TVRFNU1ERmFNSUdWTVFzd0NRWURWUVFHRXdKVlV6RVRNQkVHQTFVRUNBd0tRMkZzYVdadmNtNXBZVEVTTUJBR0ExVUVCd3dKVTJGMWMyRnNhWFJ2TVIwd0d3WURWUVFLREJSRmVHRnRjR3hsSUU5eVoyRnVhWHBoZEdsdmJqRVpNQmNHQTFVRUF3d1FTbTlvYmlCSVlXNWpiMk5yTENCTlJERWpNQ0VHQ1NxR1NJYjNEUUVKQVJZVWFtaGhibU52WTJ0QVpYaGhiWEJzWlM1dmNtY3dnZ0dpTUEwR0NTcUdTSWIzRFFFQkFRVUFBNElCandBd2dnR0tBb0lCZ1FDMExxSERGaXpKR3RveUE2OFQxTWJiT3crQXlWNVRYZ0VFNUhEb2FpaGtseFBHTFVncVlG

#### 3.4. Option to Create "detached-content" payload by removing the payload from the JWS

note the signature is displayed with the parts labeled and separated with line breaks for easier viewing then as compact serialization format

In [63]:
detached = False
if detached:
  split_sig = signature.split('.')
  split_sig[1] = ''
  signature = '.'.join(split_sig)
  for i,j in enumerate(signature.split('.')):
      print(f'{labels[i]}:\n{j}\n')
print(f'\nSignature in compact serialization format:\n{"="*80}\n{signature}')


Signature in compact serialization format:
eyJhbGciOiJSUzI1NiIsImtpZCI6IjY3NTFBNTg0NzlGMDhEQTRCNDA3NTUyODQ3MzdCNDI3OERGNEI3RUYiLCJrdHkiOiJSUyIsInNpZ1QiOiIyMDIwLTEwLTIzVDA0OjU0OjU2LjA0OCswMDowMCIsInR5cCI6IkpXVCIsIng1YyI6WyJNSUlGV2pDQ0E4S2dBd0lCQWdJVVBBME12ampLR1UzdUx6OXJOa1FmVmtPbWtTVXdEUVlKS29aSWh2Y05BUUVMQlFBd2daVXhDekFKQmdOVkJBWVRBbFZUTVJNd0VRWURWUVFJREFwRFlXeHBabTl5Ym1saE1SSXdFQVlEVlFRSERBbFRZWFZ6WVd4cGRHOHhIVEFiQmdOVkJBb01GRVY0WVcxd2JHVWdUM0puWVc1cGVtRjBhVzl1TVJrd0Z3WURWUVFEREJCS2IyaHVJRWhoYm1Odlkyc3NJRTFFTVNNd0lRWUpLb1pJaHZjTkFRa0JGaFJxYUdGdVkyOWphMEJsZUdGdGNHeGxMbTl5WnpBZUZ3MHlOVEEyTVRreU1URTVNREZhRncweU56QTJNRGt5TVRFNU1ERmFNSUdWTVFzd0NRWURWUVFHRXdKVlV6RVRNQkVHQTFVRUNBd0tRMkZzYVdadmNtNXBZVEVTTUJBR0ExVUVCd3dKVTJGMWMyRnNhWFJ2TVIwd0d3WURWUVFLREJSRmVHRnRjR3hsSUU5eVoyRnVhWHBoZEdsdmJqRVpNQmNHQTFVRUF3d1FTbTlvYmlCSVlXNWpiMk5yTENCTlJERWpNQ0VHQ1NxR1NJYjNEUUVKQVJZVWFtaGhibU52WTJ0QVpYaGhiWEJzWlM1dmNtY3dnZ0dpTUEwR0NTcUdTSWIzRFFFQkFRVUFBNElCandBd2dnR0tBb0lCZ1FDMExxSERGaXpKR3RveUE2OFQxTWJiT3cr

### 4. base64 the JWS and add the Signature element to the Bundle or QR

this is what would be contained and/or referenced by TASK over-the-wire

In [ ]:
b64_jws = b64encode(signature.encode()).decode()
sig_element = {
            "type": [  # Signature.type = Verification Signature
              {
                "system": "urn:iso-astm:E1762-95:2013",
                "code": "1.2.840.10065.1.12.1.5",
                "display": "Verification Signature"
              }
            ],
            "when": get_timestamp(timezone) if auto_timestamp else '2020-10-23T04:54:56.048+00:00', #system timestamp when signature created
            # "who" { #Reference to the Practitioner who signed and attested to the Bundle
            #   "reference": "Practitioner/123"  
            #   "display": "Practitioner/123" 
            # },
            # "onBehalfOf": { #Reference to the Organization
            #   "reference": "Organization/123"
            #   "display": "Organization/123"
            # },
            "who": { #Reference to the Practitioner who signed and attested to the Bundle using NPI should be same as SAN
                    "identifier": {
                        "system": "http://hl7.org/fhir/sid/us-npi",
                        "type" : {
                            "coding" : [{
                                "system" : "http://terminology.hl7.org/CodeSystem/v2-0203",
                                "code" : "NPI"
                              }]
                          },
                        "value": "9941339108"  # TODO use pytbon library to extract SAN
                    },
                    "display": "Dr Signer" # TODO use pytbon library to extract CN
                },
            "onBehalfOf": {
                    "identifier": {
                        "system": "http://hl7.org/fhir/sid/us-npi",
                        "value": "1184932014"
                    }
                },
            "targetFormat" : "application/fhir+json;canonicalization=http://hl7.org/fhir/canonicalization/json#document", # The technical format of the signed resources see 
            #https:// hl7.org/fhir/json.html#canonical. The MIME types can include optional parameters in the format type/subtype;
            # parameter=value, as defined in RFC 2045 and RFC 6838. For example, text/plain;charset=utf-8 specifies a character encoding.
            "sigFormat" : "application/jose", # The technical format of the signature
            "data": b64_jws,
             }

if my_obj_id :
  my_obj['id'] =  my_obj_id # update id
if my_obj_meta:
  my_obj['meta'] = my_obj_meta # update meta
if sig_element:
  my_obj['signature'] = sig_element # update signature

if my_obj['resourceType'] == 'QuestionnaireResponse':
  try:
    my_obj['extension'].append(dict(
        url='http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature',
        valueSignature=sig_element
        )) # update signature  
  except KeyError:
    my_obj['extension'] = [dict(
        url='http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature',
        valueSignature=sig_element
        )]

print(dumps(my_obj, indent=2, sort_keys=False))

{
  "resourceType": "Bundle",
  "identifier": {
    "system": "urn:ietf:rfc:3986",
    "value": "urn:uuid:c173535e-135e-48e3-ab64-38bacc68dba8"
  },
  "type": "document",
  "timestamp": "2021-10-25T20:16:29-07:00",
  "entry": [
    {
      "fullUrl": "urn:uuid:17a80a8d-4cf1-4deb-a1fd-2db1130e5f76",
      "resource": {
        "resourceType": "Composition",
        "id": "17a80a8d-4cf1-4deb-a1fd-2db1130e5f76",
        "text": {
          "status": "generated",
          "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><a name=\"Composition_17a80a8d-4cf1-4deb-a1fd-2db1130e5f76\"> </a><p class=\"res-header-id\"><b>Generated Narrative: Composition 17a80a8d-4cf1-4deb-a1fd-2db1130e5f76</b></p><a name=\"17a80a8d-4cf1-4deb-a1fd-2db1130e5f76\"> </a><a name=\"hc17a80a8d-4cf1-4deb-a1fd-2db1130e5f76\"> </a><p><b>status</b>: Final</p><p><b>type</b>: <span title=\"Codes:{http://loinc.org 11503-0}\">Medical records</span></p><p><b>encounter</b>: <a href=\"Bundle-cdex-document-digital-sig-example.h

### Using FHIR RESTful create POST to FHIR Server 
***Deactivated until until get new AIDBOX***
using AIDBox for now at `https://argopatientlist.aidbox.app/fhir/`
<!-- using `http://test.fhir.org/r4/` -->


### Alternatively Write to Local File as JSON

In [65]:
out_path = Path(r'out_files/signed_object.json')
print(f'Writing signed object to {out_path}')
out_path.write_text(dumps(my_obj,indent=2,sort_keys=False))

Writing signed object to out_files/signed_object.json


48596